In [1]:
import pandas as pd
import numpy as np
from codebase.file_utils import (
    save_obj,
    load_obj,
)
from codebase.plot import plot_density, plot_histogram, plot_line, get_post_df
from codebase.post_process import remove_cn_dimension
from codebase.mcda import get_scores_array, get_scores_df, get_prob
from codebase.data import get_final_dataset
import altair as alt
from codebase.classes_data import Data
from codebase.file_utils import (
    save_obj,
    load_obj,
    make_folder,
    path_backslash
)
from pdb import set_trace

alt.data_transformers.disable_max_rows()


DataTransformerRegistry.enable('default')

In [2]:
log_dir1 = 'log/20210425_122906_mcmc_realdata_1_ppp_6/'
exp_data1 = load_obj( 'group_data', log_dir1)
mcmcps = load_obj('ps_all_groups', log_dir1)

for name in mcmcps.keys():
    mcmcps[name] = remove_cn_dimension(mcmcps[name])

In [3]:
compute_results(mcmcps)

NameError: name 'compute_results' is not defined

In [38]:
def get_seq_density_plot(df):
    df_avg = df.groupby(['group','t'])[['value']].mean().reset_index()
    df_avg.rename({'value':'avg'}, axis=1, inplace=True)

    df_quant = df.groupby(['group','t'])[['value']].quantile(0.025).reset_index()
    df_quant.rename({'value':'q1'}, axis=1, inplace=True)
    df_quant2 = df.groupby(['group','t'])[['value']].quantile(0.975).reset_index()
    df_quant2.rename({'value':'q2'}, axis=1, inplace=True)

    dfplot = df_avg.merge(
        df_quant,on=['group','t']
        ).merge(
        df_quant2, on=['group','t']
    )


    width = 600
    height = 300
    yrange = [50,100]
    area = alt.Chart(dfplot).mark_area(
        opacity=0.2
    ).encode(
        alt.Y('q1',
              axis=alt.Axis(title='MCDA Population Score'),
              scale=alt.Scale(domain=yrange)),
        alt.Y2('q2'),
        alt.Color('group'),
        alt.X('t:O', title='patient index')
    ).properties(width=width, height=height)

    line = alt.Chart(dfplot).mark_line(
        opacity=0.9
    ).encode(
        alt.Y('avg',scale=alt.Scale(domain=yrange)),
        alt.Color('group'),
        alt.X('t:O')
    ).properties(width=width, height=height)

    return (line+area).interactive()



In [39]:
log_dir3 = 'log/smc-real-data/20210427_134915_1000particles/'
log_dir3 = 'log/20210507_135127_newweights/'
init_t = 0

time_so_far = load_obj('t', log_dir3)
print(time_so_far)
particles = load_obj('particles', log_dir3)
scores = particles.population_scores

FileNotFoundError: [Errno 2] No such file or directory: 'log/20210507_135127_newweights/t.p'

In [40]:
# log_dir3 = 'log/20210507_135127_newweights/'
# log_dir3 = 'log/20210427_083537_noinit//'
# log_dir3 = 'log/20210427_092622_init_60/'
# log_dir3 = 'log/20210427_134915_db/'
log_dir3 = 'log/smc-real-data/20210513_074220_newweights/'

init_t = 0

time_so_far = load_obj('t', log_dir3)
print(time_so_far)
particles = load_obj('particles', log_dir3)
scores = particles.population_scores

447


In [41]:
dfs = []
for t in np.linspace(0, time_so_far, 20).astype(int):
    df_t = get_scores_df(scores[t],["AVM", "MET", "RSG"]).melt(var_name = 'group')
    df_t['t'] = t+40
    dfs.append(df_t)

df = pd.concat(dfs)
get_seq_density_plot(df)

alt.LayerChart(...)

In [24]:
width = 600
height = 300
c = alt.Chart(df[df.t==time_so_far+40]).transform_fold(
    ['value']
    ).transform_density(
        density='value',
        groupby=['group'],
    ).mark_area(opacity=0.5).encode(
        alt.X('value:Q', title=None),
        alt.Y('density:Q'),
        alt.Color('group')
    ).resolve_scale(
        x='independent'
    ).properties(width=width, height=height)
c

alt.Chart(...)

In [28]:
probs1 =[]
probs2 =[]

times = np.linspace(0, time_so_far, 20).astype(int)
# times = np.arange(time_so_far)

print(times)
for t in times:
    df_t = get_scores_df(scores[t],["AVM", "MET", "RSG"])
    p1 = get_prob(df_t, 'RSG', 'AVM')
    p2 = get_prob(df_t, 'MET', 'AVM')   
    probs1.append(p1)
    probs2.append(p2) 
probdf = pd.DataFrame(probs1, columns=['RSG'])
probdf['MET'] = probs2
probdf['patient index'] = times+init_t

[  0  23  47  70  94 117 141 164 188 211 235 258 282 305 329 352 376 399
 423 447]


In [29]:
prob_plot = probdf.melt(id_vars='patient index', var_name='group', value_name='P_AVM>group')

In [30]:
prob_line = alt.Chart(prob_plot).mark_line(
    point=True
    ).encode(
        alt.Y('P_AVM>group'),
        alt.X('patient index:O'),
    alt.Color('group')
    ).properties(width=width, height=height)
prob_line

alt.Chart(...)

In [35]:
probs1 =[]
probs2 =[]

times = np.linspace(0, time_so_far, 20).astype(int)
# times = range(time_so_far)
# times = np.arange(time_so_far)


print(times)
for t in times:
    df_t = get_scores_df(scores[t],["AVM", "MET", "RSG"])
    p1 = get_prob(df_t, 'MET', 'RSG')
    probs1.append(p1)
probdf = pd.DataFrame(probs1, columns=['P(RSG>MET)'])
probdf['patient index'] = times+init_t
prob_plot = probdf
prob_line = alt.Chart(prob_plot).mark_line(
    point=True
    ).encode(
        alt.Y('P(RSG>MET)'),
        alt.X('patient index:O')
    ).properties(width=width, height=height)
prob_line

[  0  23  47  70  94 117 141 164 188 211 235 258 282 305 329 352 376 399
 423 447]


alt.Chart(...)